In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

alphapose_path = '/content/AlphaPose'
if not os.path.exists(alphapose_path):
    # clone alphapose
    #!git clone https://github.com/MVIG-SJTU/AlphaPose.git
    !git clone https://github.com/WildflowerSchools/AlphaPose.git
    
    # setting
    !export PATH=/usr/local/cuda/bin/:$PATH
    !export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    !pip install cython
    !pip install -U PyYaml tqdm
    !sudo apt-get install libyaml-dev python3-dev
    ################Only For Ubuntu 18.04#################
    !sudo locale-gen en_US.ISO-8859-1
    !sudo locale-gen C.UTF-8
    # if locale-gen not found
    !sudo apt-get install locales
    !export LANG=C.UTF-8
    ######################################################
    print('setup start')
    !cd AlphaPose && python setup.py build develop


Cloning into 'AlphaPose'...
remote: Enumerating objects: 2374, done.
remote: Total 2374 (delta 0), reused 0 (delta 0), pack-reused 2374
Receiving objects: 100% (2374/2374), 108.42 MiB | 27.90 MiB/s, done.
Resolving deltas: 100% (1218/1218), done.
     |████████████████████████████████| 645kB 6.8MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
Suggested packages:
  libyaml-doc
The following NEW packages will be installed:
  libyaml-dev
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 55.7 kB of archives.
After this operation, 246 k

In [3]:
def download_from_google_drive(file_id, file_name):
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    
    if not os.path.exists(file_name):
        # download a file from the Google Drive link
        !rm -f ./cookie
        !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
        confirm_text = !awk '/download/ {print $NF}' ./cookie
        confirm_text = confirm_text[0]
        !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name

In [4]:
yolo_pretrained_model_path = os.path.join(alphapose_path, 'detector/yolo/data/yolov3-spp.weights')
download_from_google_drive('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', yolo_pretrained_model_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1133      0 --:--:-- --:--:-- --:--:--  1133
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  240M    0  240M    0     0  98.6M      0 --:--:--  0:00:02 --:--:--  172M


In [5]:
tracking_model_path = os.path.join(alphapose_path, 'detector/tracker/data/jde.1088x608.uncertainty.pt')
download_from_google_drive('1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA', tracking_model_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2279      0 --:--:-- --:--:-- --:--:--  2266
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  557M    0  557M    0     0   130M      0 --:--:--  0:00:04 --:--:--  160M


In [7]:
# trained_model = os.path.join(alphapose_path, 'pretrained_models/simple_res50_256x192.pth')
# download_from_google_drive('1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw', trained_model)
trained_model = os.path.join(alphapose_path, 'pretrained_models/hrnet_w32_256x192.pth')
download_from_google_drive('1i63BPlOnp2vSjIZ7ni4Yp3RCPQwqe922', trained_model)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2113      0 --:--:-- --:--:-- --:--:--  2113
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  109M    0  109M    0     0  57.0M      0 --:--:--  0:00:01 --:--:--  130M


In [8]:
# cfg_file = os.path.join(alphapose_path, 'configs/coco/resnet/256x192_res50_lr1e-3_1x-simple.yaml')
cfg_file = os.path.join(alphapose_path, 'configs/coco/hrnet/256x192_w32_lr1e-3.yaml')

In [10]:
video_path = 'drive/MyDrive/video/20210304/pass1.mp4'
!cd $alphapose_path &&\
python3 scripts/demo_inference.py --cfg $cfg_file --checkpoint $trained_model --video ../$video_path --outdir ../ --save_video --vis_fast --eval

Loading pose model from /content/AlphaPose/pretrained_models/hrnet_w32_256x192.pth...
Loading YOLOv3 model..
  2% 37/1863 [00:05<04:33,  6.67it/s]Could not find encoder for codec id 27: Encoder not found
Try to use other video encoders...
 93% 1730/1863 [03:35<00:18,  7.07it/s]===========================> This video get 1782 frames in total.
 96% 1782/1863 [03:39<00:09,  8.14it/s]
===========================> Finish Model Running.
===========================> Rendering remaining 0 images in the queue...
Results have been written to json.
